## to use pyAudioAnalysis, you will need the following
  ```bash
  pip install numpy matplotlib scipy sklearn hmmlearn simplejson eyed3 pydub  
  brew install ffmeg
  (Karen, did you mean ffmpeg above?)
  ```
 ### in this github, I have already  clean up the code , skip the next steps
 
 
 #### if you are not using our github repo , but recreating in your working folder
 ```bash
 git clone https://github.com/tyiannak/pyAudioAnalysis.git
 ```
 (actually this folder was not pushed to git)
 
 #### If you are using python3, but their codes are all python 2 ( with a lot of annoying print errors)you will need this to turn their code form python2 to python 3 version
 
 #### in your working folder, run
 ```bash
2to3 -w pyAudioAnalysis
```



# you should change all the path in this block


In [2]:

# the directory should store the original mp4 video files
mp4_dirpath =  '../data/train-4/'
# this  directory will be created by the code to store the itermediate .wav file, will be cleaned up
wav_dirpath =  '../data/train-4/audio/'
# this directory will be created by the code to store the final [filename].wav.csv file for audio features
audiofeat_dirpath= '../data/train-4/audiofeat/'

numPartition = 10

# need this to upload the libraries. 
import sys
sys.path.append('./')
sys.path.append('./pyAudioAnalysis/')

In [3]:
import scipy.io.wavfile as wav
import subprocess
import os
from os import listdir
from os.path import isfile, join
import numpy as np
from pyAudioAnalysis import audioFeatureExtraction
from pyAudioAnalysis import audioBasicIO

# note, something in matplotlib will fail if don't enable X11 forwarding

In [4]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  
        if os.path.isdir(path):
            pass
        else:
            raise
def extractWavFilefromMP4(mp4_dirpath,wav_dirpath):
    mkdir_p(wav_dirpath)
    print('processing .mp4 files in ', mp4_dirpath, ' to .wav files and store in ', wav_dirpath)
    allfiles = [f for f in listdir(mp4_dirpath) if (f != '.' and f != '..')]
    for anyfile in allfiles:
        filepath=  join(mp4_dirpath, anyfile)
        if(isfile(filepath)):
            if(filepath.lower().endswith('.mp4')):
                path, filename = os.path.split(filepath)
                command = "ffmpeg -i " + filepath + " -ab 160k -ac 2 -ar 44100 -vn " + join(wav_dirpath, filename) + ".wav"
                subprocess.call(command, shell = True)
                
    print( len(allfiles), ' mp4 -> wav  Done. ')           
    
def extractAudioFeaturesfromWav(wav_dirpath,audiofeat_dirpath,numPartition):
    mkdir_p(audiofeat_dirpath)
    print('extracting audio features from .wav files into ',numPartition,' partitions', 'and store as .wav.csv files in ', audiofeat_dirpath)
    allfiles = [f for f in listdir(wav_dirpath) if (f != '.' and f != '..')]
    for anyfile in allfiles:
        filepath=  join(wav_dirpath, anyfile)
        if(isfile(filepath)):
            if(filepath.lower().endswith('.wav')):
                path, filename = os.path.split(filepath)
                (rate, sig) = audioBasicIO.readAudioFile(filepath);
                mid_range =sig.shape[0]/float(rate)/(numPartition )
                short_range = 0.05
                audioFeatureExtraction.mtFeatureExtractionToFile(filepath,mid_range,mid_range,short_range,short_range,\
                                    join(audiofeat_dirpath, filename), storeStFeatures=True, storeToCSV=True)
            
    print( len(allfiles), 'wav -> audio_feat Done. ')



In [ ]:
extractWavFilefromMP4(mp4_dirpath,wav_dirpath)
extractAudioFeaturesfromWav(wav_dirpath,audiofeat_dirpath,numPartition)
subprocess.call("rm -f " + audiofeat_dirpath + "*.npy", shell=True)  # clean up intermediate files
subprocess.call("rm -f " + audiofeat_dirpath + "*.wav_st.csv", shell=True)  # clean up intermediate files
subprocess.call("rm -rf " + wav_dirpath, shell=True)  # clean up all  wav files to save space

#subprocess.call("rm -rf " + mp4_dirpath, shell=True)  # clean up all processed mp4 files to save space

processing .mp4 files in  ../data/train-4/  to .wav files and store in  ../data/train-4/audio/
961